<a href="https://colab.research.google.com/github/Phanuwat-S-S/229351-StatisticalLearning/blob/main/229351_LAB06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ปฏิบัติการครั้งที่ 5 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้น share แล้วส่ง link มาใน mango.cmu.ac.th

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้
https://donlapark.pages.dev/229351/data/Auto.csv
หรือรันโค้ดข้างล่าง

In [1]:
!wget https://donlapark.pages.dev/229351/data/Auto.csv

--2025-10-31 09:02:39--  https://donlapark.pages.dev/229351/data/Auto.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.47.56, 172.66.44.200, 2606:4700:310c::ac42:2f38, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.47.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18079 (18K) [text/csv]
Saving to: ‘Auto.csv’

Auto.csv            100%[===================>]  17.66K  --.-KB/s    in 0s      

2025-10-31 09:02:40 (93.2 MB/s) - ‘Auto.csv’ saved [18079/18079]



In [2]:
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])
predictor_names = auto_df.columns

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### ตัวแปรต่าง ๆ มีดังนี้
mpg

    miles per gallon
cylinders

    Number of cylinders between 4 and 8
displacement

    Engine displacement (cu. inches)
horsepower

    Engine horsepower
weight

    Vehicle weight (lbs.)
acceleration

    Time to accelerate from 0 to 60 mph (sec.)
year

    Model year (modulo 100)
origin

    Origin of car (1. American, 2. European, 3. Japanese)
name

    Vehicle name


In [4]:
auto_df = auto_df.drop(['name', 'origin'],axis=1)

auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


## 1. Multivariate linear regression

เปลี่ยน $\texttt{dataframe}$ ให้เป็น numpy array ด้วย `dataframe.to_numpy()`

1. Response: $\texttt{y}$ เป็นคอลัมน์เวกเตอร์ของ `mpg`
2. Predictors: $\texttt{X}$ เป็นเมทริกซ์ของตัวแปรที่เหลือ (ใช้ `auto_df.drop`)


In [5]:
y = auto_df['mpg'].to_numpy()
X = auto_df.drop(['mpg'],axis=1).to_numpy()

คอลัมน์แรกของ $\texttt{X}$ ต้องเป็น $(1, 1, \ldots ,1 )^T$ (จำนวนเลข 1 เท่ากับจำนวนแถวของ $\texttt{X}$)  
สร้างเวกเตอร์ที่มีแต่เลข 1 และมีความยาวที่เหมาะสมด้วยคำสั่งใดคำสั่งหนึ่งต่อไปนี้  
1. $\texttt{[1]*X.shape[0]}$  
2. $\texttt{np.ones(X.shape[0])}$  

In [6]:
X.shape

(392, 6)

In [7]:
ones =  [1]*X.shape[0] #your answer here

# ใส่ ones ให้เป็นคอลัมน์แรกของ X
X = np.c_[ones,X]

In [8]:
#เช็คคำตอบ
print(X)

[[1.000e+00 8.000e+00 3.070e+02 ... 3.504e+03 1.200e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.500e+02 ... 3.693e+03 1.150e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.180e+02 ... 3.436e+03 1.100e+01 7.000e+01]
 ...
 [1.000e+00 4.000e+00 1.350e+02 ... 2.295e+03 1.160e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.200e+02 ... 2.625e+03 1.860e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.190e+02 ... 2.720e+03 1.940e+01 8.200e+01]]


### Exercise 1: จงเขียนฟังก์ชันต่อไปนี้
1. `linear_model` มี input เป็นแมทริกซ์ `X` เวกเตอร์ `y` และ output เป็นเวกเตอร์ของสัมประสิทธิ์
2. `predict` มี input เป็นแมทริกซ์ `X` เวกเตอร์ของสัมประสิทธิ์ `beta` และ output เป็นเวกเตอร์ของค่าทำนาย `y_pred`

1. Matrix multiplication $AB = \texttt{A@B}$
2. Transpose: $X^T = \texttt{X.T}$
3. Inverse: $X^{-1} = \texttt{np.linalg.inv(X)}$

$$ \hat{\beta} = (X^TX)^{-1}X^Ty $$

In [50]:
# Let's start by implementing linear regression from scratch
# using the formula provided in the class

def linear_model(X, y):
    """X: เมทริกซ์ของตัวแปรต้น"""
    """y: เวกเตอร์ของตัวแปรตาม"""
    """Return: เวกเตอร์ของ parameter beta จากการสร้าง linear regressions model
    ด้วย Ordinary Least Squares (OLS)"""
    X = np.array(X)
    y = np.array(y)

    # เพิ่มคอลัมน์ของเลข 1 เข้าไปใน X เพื่อคำนวณค่า intercept (beta_0)
    # ใช้ X.shape เพื่อให้ได้จำนวนเต็มสำหรับ np.ones
    ones_column = np.ones((X.shape[0], 1))
    X_with_intercept = np.concatenate((ones_column, X), axis=1)

    # คำนวณ beta โดยใช้สูตร OLS matrix: beta_hat = (X^T * X)^{-1} * X^T * y

    # 1. คำนวณ X^T * X
    # ใช้ .T สำหรับ transpose และ @ สำหรับ matrix multiplication
    XtX = X_with_intercept.T @ X_with_intercept

    # 2. คำนวณ inverse ของ (X^T * X)
    # ใช้ np.linalg.inv()
    # หากเมทริกซ์ singular จะเกิด LinAlgError
    XtX_inv = np.linalg.inv(XtX)

    # 3. คำนวณ beta ทั้งหมด
    beta = XtX_inv @ X_with_intercept.T @ y


    return beta


def predict(X, beta):
    """beta: array ของสัมประสิทธิ์"""
    """X: เมทริกซ์ของตัวแปรต้น"""
    """Return: ค่าทำนาย y_pred = X*beta """
    # TODO: your code here
    X = np.array(X)
    beta = np.array(beta)

    # เพิ่มคอลัมน์ 1s สำหรับ intercept เข้าไปใน X ก่อนทำนาย
    # ใช้ X.shape เพื่อให้ได้จำนวนเต็มสำหรับ np.ones
    ones_column = np.ones((X.shape[0], 1))
    X_with_intercept = np.concatenate((ones_column, X), axis=1)

    # คำนวณค่าทำนายโดยใช้ matrix multiplication
    y_pred = X_with_intercept @ beta


    return y_pred

In [51]:
beta = linear_model(X, y)
y_pred = predict(X, beta)

print("Coefficients are:", beta)
print("Predictions are:", y_pred)

Coefficients are: [ 0.56444031 -0.06484124  0.08056904 -0.12057885 -0.05163546]
Predictions are: [0.54428748 0.53710284 0.44065651 0.54948426 0.52509862 0.52295124
 0.43863569 0.46683051 0.40337511 0.47501621 0.41703563 0.45924463
 0.4608425  0.45135434 0.49078111 0.4152767  0.43463945 0.42724346
 0.371825   0.47464383 0.51926583 0.47442763 0.50730628 0.468378
 0.4755028  0.41763403 0.49266055 0.52432099 0.50351722 0.43220428
 0.45664422 0.56831267 0.48083002 0.37851211 0.44713401 0.47009726
 0.4778734  0.45719996 0.4660332  0.52751479 0.41620099 0.49247782
 0.55646679 0.43633821 0.40743825 0.57745846 0.46934441 0.59196587
 0.52532699 0.52187094 0.47225984 0.42684441 0.45640838 0.44523545
 0.39626434 0.52333037 0.40546252 0.53741764 0.49309823 0.4278666
 0.42904447 0.51970737 0.44439435 0.40187171 0.49513951 0.55131949
 0.5471545  0.41415668 0.45851698 0.47833773 0.4699769  0.51059697
 0.54348402 0.52213227 0.50540704 0.52188174 0.41079078 0.52770653
 0.41863977 0.54213239 0.47014906 0

#### จงหา $n$ (จำนวนแถวของ $\texttt{X}$) และ $p$ (จำนวนตัวแปร) ด้วย method $\texttt{X.shape[...]}$

In [52]:
n = X.shape[0]

p = X.shape[1]-1

print('Number of observations:', n)
print('Number of variables:', p)

Number of observations: 100
Number of variables: 3


## 2. F-statistic

### Exercise 2: เขียนสมมติฐานหลัก ($H_0$) และสมมติฐานแย้ง ($H_a$) ที่ใช้ F-statistic ในการทดสอบ

 H0 : ตัวแปรต้นทั้งหมดที่รวมอยู่ในโมเดลไม่มีผลกระทบต่อตัวแปรตาม
 H1 : อย่างน้อยหนึ่งในตัวแปรต้นที่รวมอยู่ในโมเดลมีอิทธิพลต่อตัวแปรตาม

### Exercise 3: จากเวกเตอร์ของ outcomes `y` คำนวณ Total Sum of Squares (TSS)

$$ \text{TSS} = \sum_{i=1}^n (y_i-\bar{y})^2 $$

$$ \text{RSS} = \sum_{i=1}^n (y_i-\hat{y})^2 $$
โดยใช้คำสั่งต่อไปนี้

$\texttt{np.sum}([y_1,y_2,...,y_n])=y_1+y_2+...+y_n$

$\texttt{np.square}([y_1,y_2,...,y_n])=[y_1^2,y^2_2,...,y^2_n]$

In [53]:
# Calculate Total Sum of Squares
y_mean = y.mean()

TSS = np.sum(np.square(y - y_mean))

RSS = np.sum(np.square(y-y_pred))

คำนวณ $F$-statistic ด้วย
$$F_{p,n-p-1} = \frac{(\text{TSS}-\text{RSS})/p}{\text{RSS}/(n-p-1)}$$
หา $n$ และจำนวนของตัวแปร $p$ ด้วย $\texttt{X.shape}[\cdot]$

### Exercise 4: คำนวณค่า F

In [54]:
F = (TSS-RSS / (p - 1)) / (RSS / (n - p))

print(F)

51.65145782255708


In [55]:
# Compute p_value of the F-statistic

p_value = stats.f.sf(F, p, n-p-1)

print(p_value)

5.793490899585343e-20


### Exercise 5: จากการทดสอบสมมติฐานที่นัยสำคัญ 0.05 เราได้ข้อสรุปอย่างไรจากค่า p-value ที่ได้ข้างบน

### คำตอบของ Exercise 5:




## 3. คำนวณ $t$-statistic

#### ขั้นตอนในการคำนวณ $t$-statistic

\begin{align*}
\text{RSE} &= \sqrt{\frac{\text{RSS}}{n-p-1}} \\
\text{SE}(\hat{\beta}) &=\text{RSE}\cdot\sqrt{\text{diag}\left( (X^TX)^{-1}\right)} \\
t &= \frac{\hat{\beta}}{\text{SE}(\hat{\beta})}
\end{align*}

In [62]:
def t_to_p_values(t_statistics, n, p):
    """
    t_statistics: numpy array ของค่า t-statistic ของตัวแปรต่างๆ
    n: จำนวนตัวอย่างข้อมูล (n_samples)
    p: จำนวนพารามิเตอร์ทั้งหมดในโมเดล (n_features + 1 สำหรับ intercept)

    Return: เวกเตอร์ของค่า p-value ของค่า t-statistic ที่ให้มา
    """
    # องศาเสรี (Degrees of Freedom)
    df = n - p
    # sf() คือ survival function (1 - CDF) คูณ 2 สำหรับ two-tailed test
    p_values = stats.t.sf(np.abs(t_statistics), df=df) * 2
    return p_values

def calculate_t_statistics_and_p_values(X_with_intercept, y, beta):
    """
    คำนวณ T-statistic และ P-value สำหรับทุกสัมประสิทธิ์ในโมเดล OLS
    """
    n = len(y)
    p = len(beta) # จำนวนพารามิเตอร์ทั้งหมด

    # 1. คำนวณ Residual Standard Error (RSE) หรือ Std Error of the Estimate
    y_pred = X_with_intercept @ beta
    residuals = y - y_pred
    rse = np.sqrt(np.sum(residuals**2) / (n - p))

    # 2. คำนวณ Covariance Matrix ของสัมประสิทธิ์ (beta)
    # Cov(beta) = RSE^2 * inv(X^T * X)
    XtX_inv = np.linalg.inv(X_with_intercept.T @ X_with_intercept)
    cov_beta = rse**2 * XtX_inv

    # 3. คำนวณ Standard Error (SE) ของแต่ละสัมประสิทธิ์ (คือ sqrt ของแนวทแยงของ Cov Matrix)
    se_beta = np.sqrt(np.diag(cov_beta))

    # 4. คำนวณ T-statistic: beta / SE(beta)
    t_statistics = beta / se_beta

    # 5. คำนวณ P-value จาก T-statistic
    p_values = t_to_p_values(t_statistics, n, p)

    return t_statistics, p_values, se_beta

### Exercise 6: จงใช้ฟังก์ชัน `t_to_p_values` ในการหาว่ามีตัวแปรใดบ้างที่มีค่า p-value น้อยกว่า 0.05 **แล้วระบุตัวแปรทั้งหมดใน text block ข้างล่าง**  

#### ใช้คำสั่ง `A.diagonal()` เพื่อดึงค่าในแนวทแยงของ `A`

In [ ]:
# TODO: your code here
diagonal_values = A.diagonal()

RSE = np.sum(np.square(y-y_pred))  # vector
SE = np.sqrt(RSE**2 * cov_beta)# vector
t =  beta / SE # vector

list ชื่อตัวแปร

In [ ]:
print("List of predictors:", predictor_names[1:-2])

### คำตอบของ Exercise 6:





## 4. Variable selection

#### statsmodels library สามารถคำนวณค่าต่างๆ เหล่านี้ได้ ผลที่แสดงจะคล้ายกับใน R  
#### มีสองวิธึในการทำ linear regression: $\texttt{statsmodels.api.sm.OLS}$
#### และ $\texttt{statsmodels.formula.api.smf.ols}$

In [43]:
import statsmodels.api as sm

X = auto_df.drop(['mpg'], axis=1)
X = sm.add_constant(X)     # add bias constant
y = auto_df['mpg']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Fri, 31 Oct 2025   Prob (F-statistic):          3.79e-135
Time:                        09:20:12   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -14.5353      4.764     -3.051   

In [44]:
import statsmodels.formula.api as smf

model2 = smf.ols('mpg ~ cylinders + displacement + horsepower + weight + acceleration + year', auto_df)
model2 = model2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Fri, 31 Oct 2025   Prob (F-statistic):          3.79e-135
Time:                        09:20:15   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.5353      4.764     -3.051   

### Exercise 7: ใช้ Backward selection ในการเลือกตัวแปร โดยที่เราจะหยุดเมื่อ p-value สของตัวแปรทุกตัว < 0.05 แล้วเขียนสมการของโมเดลที่ได้

In [ ]:
# Block for running your own code

In [ ]:
# If you want to write with for-loop

print(model2.tvalues)
print("t-value of weight:", model2.tvalues["weight"])

### คำตอบของ Exercise 7:

$$
y =
$$